# EOF Analysis of AR days

* Multivariate EOF analysis of H, U and V

In [17]:
# Import Python modules
import os, sys
from pathlib import Path
import numpy as np
import numpy.ma as ma
import pandas as  pd
import xarray as xr
from sklearn.cluster import KMeans
# matplotlib
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import AxesGrid
from matplotlib.colors import ListedColormap
from matplotlib import rcParams
import matplotlib.ticker as mticker
# cartopy
import cartopy.crs as ccrs
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
# plot styles/formatting
import seaborn as sns
import cmocean.cm as cmo
import cmocean

from IPython.display import Image, display

# Path to modules
sys.path.append('../modules')

# Import my modules
from plotter import draw_basemap
from timeseries import persistence
from eofs import *
from ar_funcs import preprocess_ar_area_subregions
from kmeans import *

In [2]:
# Set up paths

# home = Path.home()                                # users home directory
# root = home/'DATA'/'repositories'/'AR_types'      # project root directory
path_to_data = '/home/nash/DATA/data/'                 # project data -- read only
path_to_out  = '/home/nash/DATA/repositories/AR_types/out/'                          # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '/home/nash/DATA/repositories/AR_types/figs/'                        # figures

# # check that path exists
# path_to_figs.exists()

In [3]:
# Set a default font for all matplotlib text (can only set this ONCE; must restart kernel to change it)

rcParams['font.family'] = 'sans-serif'   # set the default font family to 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'    # set the default sans-serif font to 'Arial'

## Data

### AR time series

In [4]:
# read netCDF with fraction of area AR covers each subregion
filename = path_to_data + 'CH1_generated_data/ar_catalog_fraction_HASIAsubregions.nc'
ds = xr.open_dataset(filename)

# Set dates
ds = ds.sel(time=slice('1980-01-01', '2017-12-31'))

## Preprocess AR subregions - get dataframe of AR days based on area threshold
df = preprocess_ar_area_subregions(df=ds.to_dataframe(), thres=0.3)
# Show table
df.head()

,R01,R02,R03,ar,location
time,,,,,
1980-01-01,0.0,0.0,0.000000,0,NaN
1980-01-02,0.0,0.0,0.072829,0,NaN
1980-01-03,0.0,0.0,0.000000,0,NaN
1980-01-04,0.0,0.0,0.000000,0,NaN
1980-01-05,0.0,0.0,0.000000,0,NaN


### MERRA2 reanalysis

In [7]:
## Set variable names (for saving data/figs)
var_names = 'HUV500'

## Select lat/lon grid 
# # HASIA Domain
# lonmin = 0
# lonmax = 120
# latmin = 0
# latmax =  50

# Tropics/Extratropics Domain
lonmin = 0
lonmax = 120
latmin = -25
latmax = 65


### MERRA2 DATA ###
def preprocess(ds):
    '''keep only selected lats and lons'''
    return ds.sel(lat=slice(latmin, latmax), lon=slice(lonmin, lonmax))

# open H data
filepath_pattern = path_to_data + 'MERRA2/anomalies/H500/daily_*.nc'

ds_h = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, concat_dim='time', combine='by_coords')
print('ds size in GB {:0.2f}\n'.format(ds_h.nbytes / 1e9))

# # open QV data
# filepath_pattern = path_to_data + 'MERRA2/anomalies/QV500/daily_*.nc'

# ds_q = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, concat_dim='time', combine='by_coords')
# print('ds size in GB {:0.2f}\n'.format(ds_q.nbytes / 1e9))


## open UV data
filepath_pattern = path_to_data + 'MERRA2/anomalies/UV500/daily_*.nc'
ds_uv = xr.open_mfdataset(filepath_pattern, preprocess=preprocess, combine='by_coords')
print('ds size in GB {:0.2f}\n'.format(ds_uv.nbytes / 1e9))

## combine H and UV data into 1 ds object
merra = xr.merge([ds_h, ds_uv.U, ds_uv.V])
# merra

ds size in GB 3.88

ds size in GB 7.76



In [8]:
# Add AR time series to merra; set as coordinate variables
merra['ar'] = ('time', df.ar)
merra = merra.set_coords('ar')

merra['location'] = ('time', df.location)
merra = merra.set_coords('location')

# print dataset
print(merra)

<xarray.Dataset>
Dimensions:    (lat: 181, lon: 193, time: 13880)
Coordinates:
    lev        float64 500.0
  * lon        (lon) float64 0.0 0.625 1.25 1.875 ... 118.1 118.8 119.4 120.0
  * lat        (lat) float64 -25.0 -24.5 -24.0 -23.5 ... 63.5 64.0 64.5 65.0
  * time       (time) datetime64[ns] 1980-01-01T09:00:00 ... 2017-12-31T09:00:00
    dayofyear  (time) int64 dask.array<chunksize=(366,), meta=np.ndarray>
    ar         (time) int64 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0
    location   (time) object nan nan nan nan nan nan ... nan nan nan nan nan nan
Data variables:
    H          (time, lat, lon) float64 dask.array<chunksize=(366, 181, 193), meta=np.ndarray>
    U          (time, lat, lon) float64 dask.array<chunksize=(366, 181, 193), meta=np.ndarray>
    V          (time, lat, lon) float64 dask.array<chunksize=(366, 181, 193), meta=np.ndarray>


### Data Subset Selection

In [9]:
# Trim date range
start_date = '1980-12-01'
end_date = '2017-02-28'
idx = slice(start_date, end_date)
merra = merra.sel(time=idx)

# Select DJF months
idx = (merra.time.dt.month >= 12) | (merra.time.dt.month <= 2)
merra = merra.sel(time=idx)

# Select AR days JUST IN R01
idx = (merra.ar >= 1) & (merra.location == 'R01')
merra_ar = merra.sel(time=idx)

# print results
print(merra_ar)

<xarray.Dataset>
Dimensions:    (lat: 181, lon: 193, time: 213)
Coordinates:
    lev        float64 500.0
  * lon        (lon) float64 0.0 0.625 1.25 1.875 ... 118.1 118.8 119.4 120.0
  * lat        (lat) float64 -25.0 -24.5 -24.0 -23.5 ... 63.5 64.0 64.5 65.0
  * time       (time) datetime64[ns] 1980-12-12T09:00:00 ... 2017-02-18T09:00:00
    dayofyear  (time) int64 dask.array<chunksize=(2,), meta=np.ndarray>
    ar         (time) int64 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1
    location   (time) object 'R01' 'R01' 'R01' 'R01' ... 'R01' 'R01' 'R01' 'R01'
Data variables:
    H          (time, lat, lon) float64 dask.array<chunksize=(2, 181, 193), meta=np.ndarray>
    U          (time, lat, lon) float64 dask.array<chunksize=(2, 181, 193), meta=np.ndarray>
    V          (time, lat, lon) float64 dask.array<chunksize=(2, 181, 193), meta=np.ndarray>


In [10]:
# Count number of independent AR events

years = np.arange(1980, 2018) 
nyrs = len(years)
total_events = 0
for k in range(nyrs-1):    
    # Extract single DJF season
    date1 = "{}-12-01".format(years[k])
    date2 = "{}-02-28".format(years[k+1])
    x = merra.ar.sel(time=slice(date1,date2)).values
    # Count AR events in that season
    tags, tmp = persistence(x)
    # Add to running event count
    total_events += tmp

print("Number of independent AR events: ", total_events)

Number of independent AR events:  215


## Preprocessing

### Reshape, center, and standardize data matrix

In [11]:
%%time
# Load merra_ar dataset into memory
merra_ar = merra_ar.load()


CPU times: user 5.67 s, sys: 3min 47s, total: 3min 53s
Wall time: 9min 57s


In [12]:
def center_data(arr_list):
    ''' Remove the mean of an array along the first dimension.
    
    If *True*, the mean along the first axis of *dataset* (the
    time-mean) will be removed prior to analysis. If *False*,
    the mean along the first axis will not be removed. Defaults
    to *True* (mean is removed).
    The covariance interpretation relies on the input data being
    anomaly data with a time-mean of 0. Therefore this option
    should usually be set to *True*. Setting this option to
    *True* has the useful side effect of propagating missing
    values along the time dimension, ensuring that a solution
    can be found even if missing values occur in different
    locations at different times.
    '''
    for i, in_array in enumerate(arr_list):
        # Compute the mean along the first dimension.
        mean = in_array.mean(axis=0, skipna=False)
        # Return the input array with its mean along the first dimension
        # removed.
        arr_list[i] = in_array - mean
    return arr_list



In [19]:
# "Tropics" Domain
tlonmin = 0
tlonmax = 120
tlatmin = -25
tlatmax =  25

# "Extratropics" Domain
etlonmin = 0
etlonmax = 120
etlatmin = 25
etlatmax =  65

## Create list of variable arrays
# Extratropic variables
var1 = merra_ar.U.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmin, etlatmax))
var2 = merra_ar.V.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmin, etlatmax))
var3 = merra_ar.H.sel(lon=slice(etlonmin, etlonmax), lat=slice(etlatmin, etlatmax))

# Tropics variables
var4 = merra_ar.U.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmin, tlatmax))
var5 = merra_ar.V.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmin, tlatmax))
# var6 = merra_ar.QV.sel(lon=slice(tlonmin, tlonmax), lat=slice(tlatmin, tlatmax))

var_list = [var1, var2, var3, var4, var5]

In [20]:
# Center the variables by removing long-term mean
var_list = center_data(var_list)

In [21]:
def spatial_weights(arr_list):
    """Spatial weights
    
    Returns a 1D array of weights equal to the sqrt of the cos of latitude.
    
    Parameters
    ----------
    arr_list : list
        list of variable arrays
  
    Returns
    -------
    weighted_arrays : list of arrays
        weights equal to the sqrt of the cosine of latitude
    
    Example
    -------
    # Apply spatial weights using xarray
    wgts = spatial_weights(lats)            # compute weights
    era['wgts'] = ('latitude', wgts)        # add `wgts` to dataset
    era['uwnd_wgt'] = era.uwnd * era.wgts   # apply wgts to data variable
    
    """
    for i, in_array in enumerate(arr_list):
        latitude = in_array.lat
        # convert lats from degrees to radians
        lat_rad = np.deg2rad(latitude)
        # compute weights
        weights = np.sqrt(np.cos(lat_rad))
        # apply spatial weights to array
        arr_list[i] = in_array*weights
    return arr_list

In [22]:
%%time
# Weight the data by the square root of the cosine of the lat
var_list = spatial_weights(var_list)

CPU times: user 162 ms, sys: 175 ms, total: 337 ms
Wall time: 333 ms


In [ ]:
def remove_missing_values(X):
    # Find the indices of values that are missing in the whole row
    nonMissingIndex = np.where(np.logical_not(np.isnan(X[0])))[0]
    # Remove missing values from the design matrix.
    dataNoMissing = X[:, nonMissingIndex]
    return nonMissingIndex, dataNoMissing

def valid_nan(in_array):
    inan = np.isnan(in_array)
    return (inan.any(axis=0) == inan.all(axis=0)).all()

def standardize_and_flatten_arrays(arr_list, mode='t'):
    
    
    for i, in_array in enumerate(arr_list):
        # Extract variable as numpy array
            var1 = in_array.values

        # Data dimensions
            ntim, nlat, nlon = var1.shape
            npts = nlat*nlon

        # Reshape into 2D arrays by flattening the spatial dimension
            tmp1 = np.reshape(var1, (ntim, npts))

        # Remove missing data
            tmp1_idx, tmp1_miss = remove_missing_values(tmp1)

        ## Test if the removal of nans was successful
            print(valid_nan(tmp1_miss))
        
        # Data dimensions with missing values removed
            ntim, npts = tmp1_miss.shape
        
        # if t-mode
            if mode == 't':
                X1 = tmp1_miss.T
        # if s-mode
            else:
                X1 = tmp1_miss

        # Standardize by columns
            x1std = np.std(X1, axis=0)
            X1s = X1 / x1std
            
            arr_list[i] = X1s

    # Combine variables into single data matrix Xs
    nvar = len(arr_list)
    # if t-mode
    if mode == 't':
        Xs = np.empty((nvar*npts,ntim))
        Xs[0:npts,:] = X1s
        Xs[npts:npts*2,:]  = X2s
        Xs[npts*2:,:]  = X3s
    
    # if s-mode
    else:
        Xs = np.empty((ntim, nvar*npts))
        Xs[:, 0:npts] = X1s
        Xs[:, npts:npts*2]  = X2s
        Xs[:, npts*2:]  = X3s
    
    print(Xs.shape)

    # Check that column means=0 and std dev=1
    test = np.mean(np.mean(Xs, axis=0))
    print("Column means: ", np.round(test,2))
    test = np.mean(np.std(Xs, axis=0))
    print("Column std: ", np.round(test,2))
    
    return Xs

In [15]:
%%time
# Center the variables by removing long-term mean
var1 = center_data(merra_ar.U)
var2 = center_data(merra_ar.V)
var3 = center_data(merra_ar.H)

# Weight the data by the square root of the cosine of the lat
wgts = spatial_weights(merra_ar.lat)
var1 = var1*wgts
var2 = var2*wgts
var3 = var3*wgts

# Extract variables as numpy arrays
var1 = var1.values
var2 = var2.values
var3 = var3.values

# Data dimensions
ntim, nlat, nlon = var1.shape
npts = nlat*nlon
nvar = 2

# Reshape into 2D arrays by flattening the spatial dimension
tmp1 = np.reshape(var1, (ntim, npts))
tmp2 = np.reshape(var2, (ntim, npts))
tmp3 = np.reshape(var3, (ntim, npts))

# Remove missing data
tmp1_idx, tmp1_miss = remove_missing_values(tmp1)
tmp2_idx, tmp2_miss = remove_missing_values(tmp2)
tmp3_idx, tmp3_miss = remove_missing_values(tmp3)

## Test if the removal of nans was successful
print(valid_nan(tmp1_miss), valid_nan(tmp2_miss), valid_nan(tmp3_miss))

## Standardize and flatten variable arrays
Xs = standardize_and_flatten_arrays(tmp1_miss, tmp2_miss, tmp3_miss, 3)
Xs_nomiss = standardize_and_flatten_arrays(tmp1, tmp2, tmp3, 3)

True True True
(213, 46899)
Column means:  0.0
Column std:  1.0
(213, 46899)
Column means:  0.0
Column std:  1.0
CPU times: user 1.27 s, sys: 381 ms, total: 1.65 s
Wall time: 1.63 s


## EOF Analysis

In [16]:
%%time
# Compute eigenvalues & eigenvectors
evals, evecs = calc_eofs(Xs)

print('Eigenvalues: ', evals.shape)
print(evals, '\n')

print('Eigenvectors: ', evecs.shape)
print(np.round(evecs, 3), '\n')

Eigenvalues:  (46899,)
[2.33323782e-310+4.63730928e-310j 4.63730928e-310+4.63730928e-310j
 2.09648107e-001+3.17796061e-001j ... 4.81070986e-001+5.74286252e-001j
 6.66205700e-001+7.58229402e-001j 8.43238066e-001+9.21142115e-001j] 

Eigenvectors:  (46899, 46899)
[[0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]] 

CPU times: user 5min 22s, sys: 3min 54s, total: 9min 16s
Wall time: 9min 11s


### Explained Variance

In [17]:
def pct_variance(eig):
    var_eig = (eig/sum(eig))*100.
    return var_eig

In [18]:
# Calculate the percent explained var by each eigenvector
pctvar = pct_variance(evals)

# Number of EOFs that explain more than 1% of the total variance
idx = pctvar[pctvar >= 1.0]
neofs = len(idx)

# print exp var >= 1.0
cumvar = np.sum(pctvar[0:neofs-1])
print(f'Cumulative variance explained by the first {neofs} EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var: neofs = 4
cumvar = np.sum(pctvar[0:3])
print(f'Cumulative variance explained by the first 4 EOFs:')
print(f'{cumvar:.2f}% \n')

# print exp var for 4 eofs
for k in range(4):
    print(f'{k+1} \t {pctvar[k]:.2f}%')

Cumulative variance explained by the first 0 EOFs:
100.02+0.00j% 

Cumulative variance explained by the first 4 EOFs:
-0.01-0.00j% 

1 	 -0.00-0.00j%
2 	 -0.00-0.00j%
3 	 -0.01-0.00j%
4 	 -0.01-0.00j%


### North Test

In [ ]:
err = north_test(evals, total_events)
upper = pctvar + err
lower = pctvar - err

print(np.round(upper[0:6],3))
print(np.round(pctvar[0:6],3))
print(np.round(lower[0:6],3))

### Fig 2: Variance

In [ ]:
# set seaborn style
sns.set()
sns.set_style("ticks", {'patch.force_edgecolor':False})

# create figure
fig, ax = plt.subplots(figsize=(6,4))

# plot data
xvals = np.arange(neofs) + 1
ax.bar(xvals, pctvar[0:neofs], yerr=err[0:neofs], 
       color='tab:blue', alpha=0.8)

# x-axis
ax.set_xlabel('EOF')
ax.set_xticks(xvals)

# y-axis
ax.set_ylabel('Explained Variance (%)')
yticks = np.arange(0,16,3)
ax.set_yticks(yticks)
ax.set_yticklabels(yticks) 

# save fig
filepath = path_to_figs + 'exp_variance_' + var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + '.png'
plt.savefig(filepath, dpi=300)

# show
plt.show()

### Loadings

In [ ]:
neofs = 19
loads = loadings(evals, evecs, neofs)

print(loads.shape)
print(np.round(loads,3))

### Save EOFs

In [ ]:
# Save eigenvalues, eigenvectors, and loadings

neofs = 4   # number of EOFs to save (evecs, loadings3)

outfile = path_to_out + 'eigenvalues_'+ var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + '.txt'
np.savetxt(outfile, evals, fmt='%.5f')

outfile = path_to_out + 'eigenvectors_'+ var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + '.txt'
np.savetxt(outfile, evecs[:,0:neofs], fmt='%.5f', delimiter=',')

outfile = path_to_out + 'loadings_'+ var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + '.txt'
np.savetxt(outfile, loads[:,0:neofs], fmt='%.4f', delimiter=',')


### PCs

In [ ]:
# Calculate principal components (spatial modes)
neofs = 19
pcs = calc_pcs(Xs_nomiss, evecs, neofs)
# pcs = calc_pcs(Xs, evecs, neofs)

In [ ]:
# Split pcs into separate arrays for each variable
tmp1 = pcs[:,0:npts]
tmp2 = pcs[:,npts:npts*2]
tmp3 = pcs[:,npts*2:]
# tmp1 = pcs_comb[:,0:nlat*nlon]
# tmp2 = pcs_comb[:,nlat*nlon:]

# Reshape spatial dim back to 2D map
pcmodes_var1 = np.reshape(tmp1, (neofs,nlat,nlon))
pcmodes_var2 = np.reshape(tmp2, (neofs,nlat,nlon))
pcmodes_var3 = np.reshape(tmp3, (neofs,nlat,nlon))
#print(pcmodes_var1.shape, pcmodes_var2.shape)

### Fig 3: Spatial Modes

In [ ]:
# Panel Plot of Spatial Modes

# number of eofs to plot
neofs = 4

# Data for plotting
lons = merra_ar.lon.data
lats = merra_ar.lat.data
udat = pcmodes_var1[0:neofs,:,:]
vdat = pcmodes_var2[0:neofs,:,:]
data = pcmodes_var3[0:neofs,:,:]

print(np.nanmin(data), np.nanmax(data))

# Set up projection
# mapcrs = ccrs.PlateCarree()
mapcrs = ccrs.NorthPolarStereo(central_longitude=60.0)
datacrs = ccrs.PlateCarree()

# Set tick/grid locations
dx = np.arange(lonmin,lonmax+20,20)
dy = np.arange(latmin,latmax+20,20)

# subtitles
eof_label = [ ]
var_label = [ ]
for k in range(neofs):
    eof_label.append("EOF{:1d}".format(k+1,))
    var_label.append("{:.2f}%".format(pctvar[k]))

In [ ]:
fig = plt.figure(figsize=(4.0 ,4.0))
fig.dpi = 300
fname = path_to_figs + 'eofs_'+ var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + 'NPS'
fmt = 'png'

for k in np.arange(len(data)):
    ax = plt.subplot(2, 2, k+1, projection=mapcrs)
#     ax.set_extent([lons.min(), lons.max(), lats.min(), 90.], crs=mapcrs)
    # Add contour fill plot
    clevs = np.arange(-25,27.5,2.5)
    cf = ax.contourf(lons, lats, data[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
    # add vectors
    ax.quiver(lons, lats, udat[k,:,:], vdat[k,:,:], transform=datacrs,
              color='black', pivot='middle', regrid_shape=30)      
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k], loc='right', fontsize=12)
    
    ax.add_feature(cfeature.COASTLINE, edgecolor='0.4', linewidth=0.3)
#     ax.add_feature(cfeature.BORDERS, edgecolor='0.4', linewidth=0.3)

    ## Add in meridian and parallels
    gl = ax.gridlines(linewidth=.25, color='black', alpha=0.7, linestyle='--')

#     gl.xlocator = mticker.FixedLocator(np.arange(-180., 200., 20))
#     gl.ylocator = mticker.FixedLocator(np.arange(20., 70., 10.))
#     gl.xformatter = LONGITUDE_FORMATTER
#     gl.yformatter = LATITUDE_FORMATTER
    
# # add colorbar [left, bottom, width, height]
ax2 = fig.add_axes([0.13, 0.05, 0.77, 0.02])
cbar = fig.colorbar(cf, cax=ax2, drawedges=True, 
                    orientation='horizontal', extendfrac='auto')
cbar.ax.tick_params(labelsize=10)
cbar.set_label('m', fontsize=10)

plt.subplots_adjust(hspace=0.3)

fig.savefig('%s.%s' %(fname, fmt), bbox_inches='tight', dpi=fig.dpi)
fig.clf()


plotFile = fname + '.png'
print(plotFile)
# display(Image(plotFile))
plt.show()

In [ ]:
# Create figure
fig = plt.figure(figsize=(10,11))
nrows = 2
ncols = 2
mapcrs = ccrs.PlateCarree()
sns.set_style('ticks')

# Set up Axes Grid
axes_class = (GeoAxes,dict(map_projection=mapcrs))
axgr = AxesGrid(fig, 111, axes_class=axes_class,
                nrows_ncols=(nrows, ncols), axes_pad = 0.55,
                cbar_location='bottom', cbar_mode='single',
                cbar_pad=0.0, cbar_size='2.5%',label_mode='')

#newcmap = cmocean.tools.crop_by_percent(cmo.matter, 15, which='max', N=None)

# Loop for drawing each plot
for k, ax in enumerate(axgr):
    ax = draw_basemap(ax, extent=[lonmin,lonmax,latmin,latmax], xticks=dx, yticks=dy)
#     ax = draw_basemap(ax, extent=None, xticks=dx, yticks=dy)
    
    # Add contour fill plot
    clevs = np.arange(-25,27.5,2.5)
    cf = ax.contourf(lons, lats, data[k,:,:], transform=datacrs,
                     levels=clevs,
                     cmap="bwr", extend='both')
    # add vectors
    ax.quiver(lons, lats, udat[k,:,:], vdat[k,:,:], transform=datacrs,
              color='black', pivot='middle', regrid_shape=20)      
    # subtitles
    ax.set_title(eof_label[k], loc='left', fontsize=12)
    ax.set_title(var_label[k], loc='right', fontsize=12)
    
# single colorbar
cb = fig.colorbar(cf, axgr.cbar_axes[0], orientation='horizontal', drawedges=True)
cb.set_label('m', fontsize=11)
cb.ax.tick_params(labelsize=10)
    
# Display figure
filepath = path_to_figs + 'eofs_'+ var_names + str(lonmin) + str(lonmax) + str(latmin) + str(latmax) + '.png'
plt.savefig(filepath, dpi=200, bbox_inches='tight')
plt.show()